## Imports

In [1]:
import numpy as np
import pandas as pd

from IPython.display import display, clear_output

In [2]:
import jsonlines

linked_wikitext_2 = "linked-wikitext-2/"
train = linked_wikitext_2+"train.jsonl"
valid = linked_wikitext_2+"valid.jsonl"
test = linked_wikitext_2+"test.jsonl"

In [3]:
import numpy as np
import pandas as pd
import gc

from IPython.display import display, clear_output

In [4]:
emb_tsv_file = "wikidata_translation_v1.tsv"

## MLM

In [5]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers.activations import GELUActivation
from transformers.modeling_outputs import MaskedLMOutput
from transformers import DataCollatorForWholeWordMask
from datasets import load_from_disk, load_dataset
import evaluate

# import transformers

In [27]:
import wandb
wandb.init(project="kg-lm-integration", entity="tanny411")

wandb: Currently logged in as: tanny411. Use `wandb login --relogin` to force relogin


In [3]:
import torch

torch.cuda.is_available()#, torch.cuda.device_count(), torch.cuda.current_device(), torch.cuda.get_device_name(0)

/home/a2khatun/.venv/analysis/lib/python3.10/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [11]:
bert_model_name = "distilbert-base-uncased" ##"bert-base-cased"
model = AutoModel.from_pretrained(bert_model_name)
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
from torch import nn
    
class BERTModified(nn.Module):
    def __init__(self, bert_model_name):
        super().__init__()

        self.base_model = AutoModel.from_pretrained(bert_model_name)
        self.config = self.base_model.config
        
        self.activation = GELUActivation() # for distilbert
        self.vocab_transform = nn.Linear(self.config.dim, self.config.dim)
        self.vocab_layer_norm = nn.LayerNorm(self.config.dim, eps=1e-12)
        self.vocab_projector = nn.Linear(self.config.dim, self.config.vocab_size)

        self.mlm_loss_fct = nn.CrossEntropyLoss()
        
        ## set to eval
        self.base_model.eval()
        
        ## freeze model
        for param in self.base_model.parameters():
            param.requires_grad = False

    def forward(
        self,
        input_ids = None,
        attention_mask = None,
        head_mask = None,
        inputs_embeds = None,
        labels = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict= None,):
        
        base_model_output = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        hidden_states = base_model_output[0]  # (bs, seq_length, dim)
        prediction_logits = self.vocab_transform(hidden_states)  # (bs, seq_length, dim)
        prediction_logits = self.activation(prediction_logits)  # (bs, seq_length, dim)
        prediction_logits = self.vocab_layer_norm(prediction_logits)  # (bs, seq_length, dim)
        prediction_logits = self.vocab_projector(prediction_logits)  # (bs, seq_length, vocab_size)

        mlm_loss = None
        if labels is not None:
            mlm_loss = self.mlm_loss_fct(prediction_logits.view(-1, prediction_logits.size(-1)), labels.view(-1))

        return MaskedLMOutput(
            loss=mlm_loss,
            logits=prediction_logits,
            hidden_states=base_model_output.hidden_states,
            attentions=base_model_output.attentions,
        )

In [13]:
BERTModified_model = BERTModified(bert_model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [15]:
from datasets import load_dataset
dataset = load_dataset("zhengxuanzenwu/wikitext-2-split-128", split="test")

Using custom data configuration zhengxuanzenwu--wikitext-2-split-128-f504347a654a9463
Found cached dataset parquet (/home/a2khatun/.cache/huggingface/datasets/zhengxuanzenwu___parquet/zhengxuanzenwu--wikitext-2-split-128-f504347a654a9463/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [16]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

tokenized_datasets

Loading cached processed dataset at /home/a2khatun/.cache/huggingface/datasets/zhengxuanzenwu___parquet/zhengxuanzenwu--wikitext-2-split-128-f504347a654a9463/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-2d367ba46d4f5d81.arrow


Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 8192
})

In [17]:
chunk_size = 128

def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Loading cached processed dataset at /home/a2khatun/.cache/huggingface/datasets/zhengxuanzenwu___parquet/zhengxuanzenwu--wikitext-2-split-128-f504347a654a9463/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-3fec34e95f2bc86d.arrow


Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 2253
})

In [18]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/a2khatun/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [19]:
train_size = 1000
test_size = 100

downsampled_dataset = lm_datasets.train_test_split(train_size=train_size, test_size=test_size, seed=42)
downsampled_dataset

Loading cached split indices for dataset at /home/a2khatun/.cache/huggingface/datasets/zhengxuanzenwu___parquet/zhengxuanzenwu--wikitext-2-split-128-f504347a654a9463/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-827befdce6c51896.arrow and /home/a2khatun/.cache/huggingface/datasets/zhengxuanzenwu___parquet/zhengxuanzenwu--wikitext-2-split-128-f504347a654a9463/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-3dd2a0e3127c60f1.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 100
    })
})

In [20]:
metric_name = evaluate.load("perplexity") # accuracy

In [32]:
from transformers import TrainingArguments

batch_size = 16

# Show the training loss with every epoch
logging_steps = len(downsampled_dataset['train']) // batch_size
model_name = "BERTModified"
output_dir = f"{model_name}-finetuned-wikitext-test"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
#     fp16=True,
    logging_steps=logging_steps,
    num_train_epochs=1,
#     load_best_model_at_end=True,
#     metric_for_best_model="loss",#metric_name,
#     greater_is_better = False,
    logging_dir='logs',
    report_to="wandb",
#     no_cuda=True,
)

PyTorch: setting up devices


metric_for_best_model (str, optional) — Use in conjunction with load_best_model_at_end to specify the metric to use to compare two different models. Must be the name of a metric returned by the evaluation with or without the prefix "eval_". Will default to "loss" if unspecified and load_best_model_at_end=True (to use the evaluation loss).

If you set this value, greater_is_better will default to True. Don’t forget to set it to False if your metric is better when lower.

In [33]:
from transformers import Trainer

trainer = Trainer(
    model=BERTModified_model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
)

/home/a2khatun/Downloads/KG/project/BERTModified-finetuned-wikitext-test is already a clone of https://huggingface.co/Aisha/BERTModified-finetuned-wikitext-test. Make sure you pull the latest changes with `repo.git_pull()`.


In [24]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

The following columns in the evaluation set don't have a corresponding argument in `BERTModified.forward` and have been ignored: word_ids. If word_ids are not expected by `BERTModified.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16


>>> Perplexity: 37705.04


In [34]:
trainer.train()
# trainer.save_model("output/models/BERTModified")

The following columns in the training set don't have a corresponding argument in `BERTModified.forward` and have been ignored: word_ids. If word_ids are not expected by `BERTModified.forward`,  you can safely ignore this message.
/home/a2khatun/.venv/analysis/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 63
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,9.086300,8.856506


The following columns in the evaluation set don't have a corresponding argument in `BERTModified.forward` and have been ignored: word_ids. If word_ids are not expected by `BERTModified.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16
Saving model checkpoint to BERTModified-finetuned-wikitext-test/checkpoint-63
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


KeyError: 'eval_perplexity'

https://theaisummer.com/hugging-face-vit/

If for example we wanted to visualize the training process using the weights and biases library, we can use the WandbCallback. We can simply add another argument to the Trainer in the form of:
```
from transformers import WandbCallback
callbacks = [WandbCallback(...)]
```
One other thing: Take a look at the logging_dir='logs'. By saving the training logs, we can very easily initiate a tensorboard instance and track the training progress:

```
$ tensorboard --logdir logs/
```

An alternative is to use the TensorBoardCallback provided by the library.

In [58]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}") #21596 for 1 epoch

The following columns in the evaluation set don't have a corresponding argument in `BERTModified.forward` and have been ignored: word_ids. If word_ids are not expected by `BERTModified.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 16


>>> Perplexity: 828.09


In [59]:
trainer.push_to_hub()

Saving model checkpoint to BERTModified-finetuned-wikitext-test
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Upload file pytorch_model.bin:   0%|          | 32.0k/345M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Aisha/BERTModified-finetuned-wikitext-test
   dbda7e7..ec41d60  main -> main

Dropping the following result as it does not have all the necessary fields:
{}
To https://huggingface.co/Aisha/BERTModified-finetuned-wikitext-test
   ec41d60..721863d  main -> main



'https://huggingface.co/Aisha/BERTModified-finetuned-wikitext-test/commit/ec41d60af7b0dbf3304bf650d114b445e1f716fa'

In [47]:
# from huggingface_hub import get_full_repo_name

# repo_name = get_full_repo_name(output_dir)
# repo_name

In [48]:
# from huggingface_hub import Repository

# repo = Repository(output_dir, clone_from=repo_name)

In [60]:
from transformers import pipeline

# Initialize MLM pipeline
mlm = pipeline('fill-mask', model=BERTModified_model, tokenizer=tokenizer)

# Get mask token
mask = mlm.tokenizer.mask_token

# Get result for particular masked phrase
phrase = f'Wikipedia is a free online {mask}, created and edited by volunteers around the world'

result = mlm(phrase)

# Print result
print(result)

[{'score': 0.016104264184832573, 'token': 1010, 'token_str': ',', 'sequence': 'wikipedia is a free online,, created and edited by volunteers around the world'}, {'score': 0.005953500047326088, 'token': 1998, 'token_str': 'and', 'sequence': 'wikipedia is a free online and, created and edited by volunteers around the world'}, {'score': 0.005101792048662901, 'token': 2000, 'token_str': 'to', 'sequence': 'wikipedia is a free online to, created and edited by volunteers around the world'}, {'score': 0.004948945250362158, 'token': 1030, 'token_str': '@', 'sequence': 'wikipedia is a free online @, created and edited by volunteers around the world'}, {'score': 0.004703795537352562, 'token': 1012, 'token_str': '.', 'sequence': 'wikipedia is a free online., created and edited by volunteers around the world'}]


In [61]:
for x in result:
    print(f">>> {x['sequence']}")

>>> wikipedia is a free online,, created and edited by volunteers around the world
>>> wikipedia is a free online and, created and edited by volunteers around the world
>>> wikipedia is a free online to, created and edited by volunteers around the world
>>> wikipedia is a free online @, created and edited by volunteers around the world
>>> wikipedia is a free online., created and edited by volunteers around the world


https://huggingface.co/course/chapter7/3?fw=pt#perplexity-for-language-models

In [78]:
## compare model weights

for p1, p2 in zip(model.parameters(), BERTModified_model.parameters()):
#     print(p1.shape, p2.shape)
    if p1.data.ne(p2.data).sum() > 0:
        print(False)

In [84]:
BERTModified_model

BERTModified(
  (base_model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_

In [86]:
params = [x for x in BERTModified_model.parameters()]
for p in params[-6:]:
    print(p.shape)

torch.Size([768, 768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([30522, 768])
torch.Size([30522])


## KG Model

In [4]:
import jsonlines

linked_wikitext_2 = "linked-wikitext-2/"
train = linked_wikitext_2+"train.jsonl"
valid = linked_wikitext_2+"valid.jsonl"
test = linked_wikitext_2+"test.jsonl"

## Create Embedding Map

In [5]:
import numpy as np
import pandas as pd
import gc

from IPython.display import display, clear_output

In [6]:
emb_tsv_file = "wikidata_translation_v1.tsv"

In [4]:
df_size = 0
ix = 0

for chunk in pd.read_csv(emb_tsv_file, 
                         delimiter='\t', 
                         header=None, 
                         chunksize=10000, 
                         skiprows=1, 
                         encoding='unicode-escape',
                         names=['id']+[f"embedding_{num}" for num in range(1,201)]):
    
    chunk = chunk.dropna()
    qid_df = chunk[chunk['id'].str.startswith("<http://www.wikidata.org/entity/")]
    df_size += len(qid_df)
    qid_df["id"] = qid_df["id"].apply(lambda x: x.split("/")[-1][:-1])
    qid_df.to_csv(f"qid_embedding.csv", index=False, header=False, mode="a")
#     qid_df.to_csv(f"qid_embeding/qid_embedding_{ix}.csv", index=False, header=False, mode="a")
    ix+=1
    
    clear_output()
    gc.collect()

print("Done!")

Done!


In [5]:
df_size

55032670

In [7]:
for embedding_qids in pd.read_csv('qid_embedding.csv', 
                         header=None, 
                         chunksize=10000, 
                         names=['id']+[f"embedding_{num}" for num in range(1,201)]):
    break

In [8]:
embedding_qids

,id,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_191,embedding_192,embedding_193,embedding_194,embedding_195,embedding_196,embedding_197,embedding_198,embedding_199,embedding_200
0,Q13442814,-0.0828,-0.1610,0.2152,-0.1867,-0.2638,0.2255,0.7521,-0.2523,-0.9443,...,0.4445,-0.2946,-0.1511,-0.4235,-0.0393,0.5657,-0.0626,-0.2530,-0.1620,0.0711
1,Q5,-0.2249,0.1400,-0.1503,-0.4524,-0.2512,0.2315,0.9779,0.6769,0.0827,...,-0.1156,0.2502,0.9211,-0.3094,-0.7469,0.4710,0.5569,-0.7302,-0.0854,0.2069
2,Q4167836,0.2119,-0.0596,-0.4983,-0.5406,-0.8534,0.1305,1.0053,0.0972,-0.6799,...,-0.0157,-0.2806,0.3098,-0.1842,-0.1467,-0.1482,0.4218,-0.5890,0.0637,0.0597
3,Q6581097,0.3376,0.2482,0.2568,-0.0287,-0.8143,-0.6013,0.4635,0.1256,-0.2780,...,0.0733,-0.0252,-0.2832,0.3008,-0.2845,0.5018,-0.3458,-0.7110,0.2288,-0.5106
4,Q16521,-0.3649,-0.7143,-0.1985,-0.8482,-0.6250,-0.1299,0.4153,-0.0469,-0.5142,...,0.7377,-0.2373,0.8955,-0.5313,0.6623,0.2243,0.6147,-0.5094,-0.2916,0.4188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Q26842633,-0.3039,-0.3113,-0.4618,-0.3129,-0.8078,0.0136,0.4512,-0.9440,0.1844,...,0.5912,0.9583,1.2572,0.5230,-0.5270,-0.3459,0.6681,-0.0368,-0.2663,-0.0909
9996,Q15763806,-1.2473,0.6681,-0.7949,-0.7073,0.1150,-0.8819,0.5306,0.4668,-0.1859,...,0.4987,0.3669,-0.1575,0.2222,0.0021,0.1813,0.4070,-0.2016,-0.2948,0.3876
9997,Q23916,0.0757,0.0808,-0.4001,-0.3316,0.1981,0.0842,0.7219,-0.0462,0.0717,...,-0.2297,-0.3597,0.4675,-0.2015,-0.0844,0.0382,0.4272,0.1239,-0.0524,0.0774
9998,Q864217,-0.5102,-0.3203,0.1878,0.0757,-0.7398,-0.1611,0.3297,0.8216,-0.7161,...,-0.1461,-0.5963,-0.2641,0.0739,-0.0134,-0.2057,-0.7483,0.1778,0.1733,0.2463


In [16]:
embedding_qids[embedding_qids['id'] == "Q5"].iloc[0,1:].values.reshape((1,200))

array([[-0.2249, 0.14, -0.1503, -0.4524, -0.2512, 0.2315, 0.9779, 0.6769,
        0.0827, 0.174, 0.0746, -0.3145, -0.4379, -0.2596, 0.2497, 0.6317,
        0.0649, 0.8525, -0.844, -0.325, 0.1749, 0.2906, -0.7133, 0.6766,
        -0.1764, 0.4321, -0.4739, -0.8006, -0.3901, 0.7154, 0.316,
        -0.0677, 0.4136, 0.016, 0.1898, -0.9656, 0.2196, -0.7675,
        -0.8107, -0.7817, 0.5434, -0.619, -0.422, -0.2386, -0.4517,
        0.1504, -0.431, -0.4969, 0.4982, -0.274, -0.55, -0.0774, -0.5733,
        0.9172, -0.118, -0.0867, -0.3846, -0.2733, -0.6376, -0.182,
        0.756, 0.3724, 0.0529, 0.261, 0.2723, 0.1296, 0.1688, 0.644,
        -0.0943, -0.6549, -0.2459, 0.1955, 0.1156, -0.0082, 0.0364,
        0.3262, 0.7579, -0.2566, -0.439, 0.0152, 0.1234, 1.1579, -0.9209,
        0.3654, -0.0719, 0.5497, -0.1902, -0.0008, -0.1432, 0.5596,
        0.5633, 0.768, 0.138, -0.2131, -0.5384, -0.0125, -0.7738, 1.1437,
        -0.1613, -0.2311, 0.2402, -0.4568, -0.752, -0.2087, 0.4484,
        -0.0869

In [12]:
qids = pd.read_csv('qid_embedding.csv', header=None, names=['id'], usecols=[0])
qids.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55032670 entries, 0 to 55032669
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      object
dtypes: object(1)
memory usage: 419.9+ MB


In [13]:
qids

,id
0,Q13442814
1,Q5
2,Q4167836
3,Q6581097
4,Q16521
...,...
55032665,Q61905263
55032666,Q61908653
55032667,Q61908654
55032668,Q61909529


In [5]:
embed_range = pd.read_csv('qid_embedding.csv', 
                          header=None, 
                          nrows=10, 
                          skiprows=55032665,
                          names=['id']+[f"embedding_{num}" for num in range(1,201)])
embed_range

,id,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_191,embedding_192,embedding_193,embedding_194,embedding_195,embedding_196,embedding_197,embedding_198,embedding_199,embedding_200
0,Q61905263,-0.2547,0.1266,0.4141,-0.1665,0.0902,-0.2426,0.0482,0.0156,-0.0167,...,-0.2337,0.0648,0.3816,0.1224,-0.1724,0.1299,0.3947,0.0028,0.2577,0.0900
1,Q61908653,-0.3424,0.0551,0.1168,-0.0450,-0.2718,-0.3656,-0.1966,-0.0885,0.1675,...,0.1896,-0.0315,0.2755,-0.0479,-0.1841,0.0546,-0.0575,-0.0443,0.1206,-0.1594
2,Q61908654,-0.2041,-0.0213,0.1148,0.1016,-0.1224,-0.1033,0.1357,0.1167,-0.1219,...,0.1765,-0.1373,0.1718,-0.1315,-0.1707,-0.3487,-0.1054,0.1045,0.1903,-0.1669
3,Q61909529,-0.1326,0.0164,-0.0338,0.0486,-0.1860,-0.4355,-0.1516,0.0393,0.1787,...,0.1099,-0.0978,0.2751,-0.1946,-0.2837,0.0414,-0.1365,0.0342,0.1047,-0.0946
4,Q61910646,-0.4570,0.3926,0.3603,0.0041,-0.2131,-0.3305,-0.3827,0.0623,-0.0138,...,0.1182,-0.1485,-0.1201,0.1351,-0.0515,-0.2484,-0.0374,0.1714,-0.0102,-0.2960


In [6]:
train, valid, test

('linked-wikitext-2/train.jsonl',
 'linked-wikitext-2/valid.jsonl',
 'linked-wikitext-2/test.jsonl')

In [11]:
## seletc Q-ids only in linked wikitext-2

import jsonlines

qid_set = set()

for dataset in [train, valid, test]:
    with jsonlines.open(dataset) as f:
        for line in f.iter():
            for annot in line['annotations']:
                qid_set.add(annot['id'])
        print(len(qid_set))

41058
44413
47932


In [16]:
qids_wktxt2 = qids[qids['id'].isin(qid_set)]

In [17]:
qids_wktxt2.to_csv("qids_wktxt2.csv", index=False)

In [26]:
qids_wktxt2 #.index.values

,id
1,Q5
4,Q16521
5,Q7432
6,Q30
7,Q1860
...,...
54913387,Q17042242
54973666,Q28208712
54991087,Q42377501
54993341,Q48816565


In [28]:
len(qids)

55032670

In [34]:
# embeds_wktxt = pd.DataFrame(columns=['id']+[f"embedding_{num}" for num in range(1,201)])

to_exclude = [i for i in range(55032670) if i not in qids_wktxt2.index.values]

embeds_wktxt = pd.read_csv('qid_embedding.csv',
                              header=None, 
                              skiprows=to_exclude,
                              names=['id']+[f"embedding_{num}" for num in range(1,201)])

len(embeds_wktxt)

46685

In [35]:
embeds_wktxt

,id,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_191,embedding_192,embedding_193,embedding_194,embedding_195,embedding_196,embedding_197,embedding_198,embedding_199,embedding_200
0,Q5,-0.2249,0.1400,-0.1503,-0.4524,-0.2512,0.2315,0.9779,0.6769,0.0827,...,-0.1156,0.2502,0.9211,-0.3094,-0.7469,0.4710,0.5569,-0.7302,-0.0854,0.2069
1,Q16521,-0.3649,-0.7143,-0.1985,-0.8482,-0.6250,-0.1299,0.4153,-0.0469,-0.5142,...,0.7377,-0.2373,0.8955,-0.5313,0.6623,0.2243,0.6147,-0.5094,-0.2916,0.4188
2,Q7432,0.2851,0.3203,0.1160,-0.2781,-0.0969,-0.4335,-0.6281,-0.4147,0.4710,...,0.3683,-0.6955,0.1620,0.1344,-0.1442,-0.0663,0.6458,-0.6264,0.8500,0.5677
3,Q30,1.0045,-0.1153,-0.0770,0.3894,-0.2120,-0.3979,0.3745,0.4687,-0.7981,...,0.3716,0.0019,-0.2455,0.5625,-0.3040,-0.4212,0.2615,-0.7009,-0.1871,-0.1948
4,Q1860,0.2808,-0.0076,0.3640,-0.3508,0.2861,0.1748,0.3726,0.5543,-0.6450,...,-0.2940,-1.1060,0.5231,-0.2903,0.2146,0.5142,0.0470,0.4795,0.0279,-0.9129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46680,Q17042242,-0.1863,-0.0469,0.0483,0.1670,-0.2604,-0.0706,0.0765,0.0135,-0.0309,...,-0.0811,0.0552,0.3624,-0.0836,-0.1808,-0.0564,-0.0138,0.1261,-0.0036,0.0071
46681,Q28208712,-0.1813,-0.0169,0.0940,0.2144,-0.1460,-0.0879,0.1475,0.0829,-0.1985,...,0.1492,-0.1119,0.2504,-0.1439,-0.1438,-0.4153,-0.1049,0.3060,0.1783,-0.1870
46682,Q42377501,-0.2616,-0.1402,0.2998,0.0265,-0.2412,-0.2058,-0.2162,-0.1520,-0.1352,...,0.1989,-0.1950,0.4726,0.0322,0.0439,-0.0669,0.0326,-0.0173,0.2337,-0.1331
46683,Q48816565,0.0007,-0.0841,-0.0816,-0.0922,-0.0922,-0.0315,0.0156,0.3363,0.2449,...,0.0873,0.2758,0.2465,0.2607,0.2390,-0.4363,0.1404,-0.1441,-0.0521,0.0599


In [36]:
embeds_wktxt.to_csv("embeds_wktxt.csv", index=False)

In [37]:
embeds_wktxt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46685 entries, 0 to 46684
Columns: 201 entries, id to embedding_200
dtypes: float64(200), object(1)
memory usage: 71.6+ MB


In [40]:
import jsonlines

with jsonlines.open(valid) as f:
    for line in f.iter():
        print(len(line['tokens']))

1352
10663
2474
10767
3619
1247
4262
1559
3602
1689
2038
1456
6072
1832
3524
18805
3843
1291
1535
1955
2173
5384
3375
875
2588
857
3129
2023
6652
6681
5720
3951
5427
2929
5950
1301
4328
4530
2648
2046
4824
1364
3407
4253
1325
1287
2078
1705
5747
1314
5267
2746
7504
2746
1887
5184
2060
772
1358
3096


In [62]:
embeds_wktxt

,id,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_191,embedding_192,embedding_193,embedding_194,embedding_195,embedding_196,embedding_197,embedding_198,embedding_199,embedding_200
0,Q5,-0.2249,0.1400,-0.1503,-0.4524,-0.2512,0.2315,0.9779,0.6769,0.0827,...,-0.1156,0.2502,0.9211,-0.3094,-0.7469,0.4710,0.5569,-0.7302,-0.0854,0.2069
1,Q16521,-0.3649,-0.7143,-0.1985,-0.8482,-0.6250,-0.1299,0.4153,-0.0469,-0.5142,...,0.7377,-0.2373,0.8955,-0.5313,0.6623,0.2243,0.6147,-0.5094,-0.2916,0.4188
2,Q7432,0.2851,0.3203,0.1160,-0.2781,-0.0969,-0.4335,-0.6281,-0.4147,0.4710,...,0.3683,-0.6955,0.1620,0.1344,-0.1442,-0.0663,0.6458,-0.6264,0.8500,0.5677
3,Q30,1.0045,-0.1153,-0.0770,0.3894,-0.2120,-0.3979,0.3745,0.4687,-0.7981,...,0.3716,0.0019,-0.2455,0.5625,-0.3040,-0.4212,0.2615,-0.7009,-0.1871,-0.1948
4,Q1860,0.2808,-0.0076,0.3640,-0.3508,0.2861,0.1748,0.3726,0.5543,-0.6450,...,-0.2940,-1.1060,0.5231,-0.2903,0.2146,0.5142,0.0470,0.4795,0.0279,-0.9129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46680,Q17042242,-0.1863,-0.0469,0.0483,0.1670,-0.2604,-0.0706,0.0765,0.0135,-0.0309,...,-0.0811,0.0552,0.3624,-0.0836,-0.1808,-0.0564,-0.0138,0.1261,-0.0036,0.0071
46681,Q28208712,-0.1813,-0.0169,0.0940,0.2144,-0.1460,-0.0879,0.1475,0.0829,-0.1985,...,0.1492,-0.1119,0.2504,-0.1439,-0.1438,-0.4153,-0.1049,0.3060,0.1783,-0.1870
46682,Q42377501,-0.2616,-0.1402,0.2998,0.0265,-0.2412,-0.2058,-0.2162,-0.1520,-0.1352,...,0.1989,-0.1950,0.4726,0.0322,0.0439,-0.0669,0.0326,-0.0173,0.2337,-0.1331
46683,Q48816565,0.0007,-0.0841,-0.0816,-0.0922,-0.0922,-0.0315,0.0156,0.3363,0.2449,...,0.0873,0.2758,0.2465,0.2607,0.2390,-0.4363,0.1404,-0.1441,-0.0521,0.0599


In [122]:
import jsonlines

for dataset, filename in zip([valid, test, train], ["valid", "test", "train"]):
    line_count = 0
    with jsonlines.open(dataset) as f:
        embed_list_dataset = []
        for line in f.iter():
            line_count += 1
            embed_list = np.zeros((len(line['tokens']), 200))
            
            for annot in line['annotations']:
                start, end = annot['span']
                qid = annot['id']
                df = embeds_wktxt[embeds_wktxt['id']==qid]
                if len(df)>0:
                    embed_list[start:end] = np.tile(
                            df.iloc[0,1:].values.reshape((1,200)),
                            (end-start, 1)
                        )
                else:
                    print(qid)
                    
            embed_list_dataset.append(embed_list)

#         with open(f'{filename}.npy', 'ab') as f:
#             np.save(f, embed_list)
        np.savez(f"{filename}.npz", *embed_list_dataset)
    
    print("\n", "="*50, filename, line_count, "\n")

T::11::+1964-06-21T00:00:00Z
T::11::+2004-09-02T00:00:00Z
V::0.8000::http://www.wikidata.org/entity/Q11229
T::9::+1970-00-00T00:00:00Z
T::9::+1970-00-00T00:00:00Z
T::11::+2002-11-22T00:00:00Z
T::11::+2002-11-22T00:00:00Z
T::11::+2002-11-22T00:00:00Z
T::11::+2002-11-22T00:00:00Z
T::11::+2002-11-22T00:00:00Z
T::9::+2008-00-00T00:00:00Z
T::11::+1986-10-07T00:00:00Z
Q4663150
T::11::+1986-10-07T00:00:00Z
T::9::+2015-00-00T00:00:00Z
T::9::+2016-00-00T00:00:00Z
T::11::+2007-09-09T00:00:00Z
T::9::+2008-00-00T00:00:00Z
T::11::+1873-04-04T00:00:00Z
T::9::+1874-00-00T00:00:00Z
T::11::+1873-04-04T00:00:00Z
T::9::+1758-00-00T00:00:00Z
T::9::+2005-01-01T00:00:00Z
T::11::+1991-06-23T00:00:00Z
T::11::+1991-06-23T00:00:00Z
T::11::+2009-09-09T00:00:00Z
T::11::+1869-08-05T00:00:00Z
T::11::+1940-01-09T00:00:00Z
T::11::+1839-08-27T00:00:00Z
T::11::+1840-09-02T00:00:00Z
T::11::+1804-03-04T00:00:00Z
T::11::+1808-01-26T00:00:00Z
T::11::+1810-01-01T00:00:00Z
T::9::+1832-00-00T00:00:00Z
T::9::+1829-01-01T00:00:

T::11::+0068-06-09T00:00:00Z
T::11::+0069-12-21T00:00:00Z
T::11::+0069-12-21T00:00:00Z
T::11::+0098-01-27T00:00:00Z
T::11::+0098-01-27T00:00:00Z
T::9::+0229-00-00T00:00:00Z
T::11::+0096-09-18T00:00:00Z
T::9::+1951-00-00T00:00:00Z
T::9::+1667-00-00T00:00:00Z
T::9::+1880-00-00T00:00:00Z
T::11::+1992-02-27T00:00:00Z
T::11::+1992-02-27T00:00:00Z
T::11::+2008-09-05T00:00:00Z
T::11::+2012-11-08T00:00:00Z
T::11::+2013-10-27T00:00:00Z
T::11::+2013-10-27T00:00:00Z
T::9::+1991-00-00T00:00:00Z
T::11::+2012-12-25T00:00:00Z
T::9::+2005-00-00T00:00:00Z
T::11::+2012-12-25T00:00:00Z
T::11::+1963-11-23T00:00:00Z
T::11::+1963-11-23T00:00:00Z
T::9::+1917-01-01T00:00:00Z
T::9::+2011-00-00T00:00:00Z
T::9::+2010-00-00T00:00:00Z
T::9::+1994-00-00T00:00:00Z
T::9::+1993-00-00T00:00:00Z
T::9::+2011-00-00T00:00:00Z
T::11::+2012-05-08T00:00:00Z
T::11::+2012-07-05T00:00:00Z
T::11::+2012-11-27T00:00:00Z
T::9::+2008-00-00T00:00:00Z
T::9::+2011-00-00T00:00:00Z
T::11::+2012-07-05T00:00:00Z
T::11::+2012-11-27T00:00:00Z

T::11::+1998-08-26T00:00:00Z
T::11::+1801-01-01T00:00:00Z
T::11::+1801-01-01T00:00:00Z
T::11::+1803-09-20T00:00:00Z
T::11::+1801-01-01T00:00:00Z
T::11::+1921-07-11T00:00:00Z
T::11::+1923-05-24T00:00:00Z
T::9::+1845-00-00T00:00:00Z
T::9::+2006-01-01T00:00:00Z
T::11::+1667-11-30T00:00:00Z
T::11::+1745-10-19T00:00:00Z
T::11::+1742-04-13T00:00:00Z
T::9::+1921-00-00T00:00:00Z
T::11::+1894-11-05T00:00:00Z
T::11::+1952-11-08T00:00:00Z
T::9::+1947-00-00T00:00:00Z
T::11::+1894-11-05T00:00:00Z
T::9::+1933-00-00T00:00:00Z
T::9::+1931-01-01T00:00:00Z
T::11::+1909-03-04T00:00:00Z
T::11::+1868-11-17T00:00:00Z
T::11::+2009-10-03T00:00:00Z
T::9::+2012-00-00T00:00:00Z
T::9::+1895-00-00T00:00:00Z
T::11::+1978-07-25T00:00:00Z
T::9::+2002-01-01T00:00:00Z
T::11::+1836-03-12T00:00:00Z
T::11::+1865-02-06T00:00:00Z
T::9::+1857-01-01T00:00:00Z
T::9::+1861-00-00T00:00:00Z
T::11::+1836-03-12T00:00:00Z
T::11::+1852-03-20T00:00:00Z
T::11::+1852-03-20T00:00:00Z
T::9::+1861-00-00T00:00:00Z
T::9::+1885-00-00T00:00:00

T::11::+1988-05-02T00:00:00Z
T::11::+1941-06-22T00:00:00Z
T::11::+1943-08-17T00:00:00Z
T::11::+1944-06-06T00:00:00Z
T::11::+1974-07-12T00:00:00Z
T::9::+1997-00-00T00:00:00Z
T::11::+2001-03-26T00:00:00Z
T::11::+2001-03-26T00:00:00Z
T::11::+1898-07-07T00:00:00Z
T::11::+1865-04-15T00:00:00Z
T::11::+1851-07-27T00:00:00Z
T::11::+1890-11-19T00:00:00Z
T::11::+1874-02-06T00:00:00Z
T::11::+1851-07-27T00:00:00Z
T::11::+1874-02-06T00:00:00Z
T::11::+1847-05-07T00:00:00Z
T::11::+1894-03-05T00:00:00Z
T::9::+1883-00-00T00:00:00Z
T::9::+1883-00-00T00:00:00Z
T::9::+1987-00-00T00:00:00Z
T::9::+1988-01-01T00:00:00Z
T::11::+2008-04-22T00:00:00Z
T::11::+1975-08-15T00:00:00Z
T::9::+1973-01-01T00:00:00Z
T::9::+1968-01-01T00:00:00Z
T::9::+1973-01-01T00:00:00Z
T::9::+1959-01-01T00:00:00Z
T::9::+1973-01-01T00:00:00Z
T::11::+1975-08-15T00:00:00Z
T::11::+1975-08-15T00:00:00Z
T::11::+2002-07-25T00:00:00Z
T::9::+1980-01-01T00:00:00Z
T::9::+1987-01-01T00:00:00Z
T::9::+1973-01-01T00:00:00Z
T::9::+2004-01-01T00:00:00Z

T::11::+1950-01-26T00:00:00Z
T::11::+1947-08-15T00:00:00Z
T::11::+1947-08-15T00:00:00Z
T::11::+1994-12-18T00:00:00Z
T::10::+2003-05-00T00:00:00Z
T::10::+2003-05-00T00:00:00Z
T::11::+1863-06-20T00:00:00Z
T::11::+2004-07-08T00:00:00Z
T::10::+2003-05-00T00:00:00Z
T::10::+2003-05-00T00:00:00Z
T::10::+2003-05-00T00:00:00Z
T::10::+2003-05-00T00:00:00Z
T::10::+2003-05-00T00:00:00Z
T::9::+2007-01-01T00:00:00Z
T::11::+1933-03-27T00:00:00Z
T::11::+1939-10-25T00:00:00Z
T::11::+1939-10-25T00:00:00Z
T::11::+1986-09-17T00:00:00Z
T::11::+1985-08-16T00:00:00Z
T::11::+1986-09-17T00:00:00Z
T::11::+1986-09-17T00:00:00Z
T::9::+1983-00-00T00:00:00Z
T::9::+1987-00-00T00:00:00Z
T::9::+1987-00-00T00:00:00Z
T::11::+1942-07-04T00:00:00Z
T::9::+1940-00-00T00:00:00Z
T::9::+1956-00-00T00:00:00Z
Q17305006
T::9::+1956-00-00T00:00:00Z
T::9::+1951-00-00T00:00:00Z
T::9::+1974-00-00T00:00:00Z
T::9::+1951-00-00T00:00:00Z
T::11::+2005-12-21T00:00:00Z
T::11::+2004-11-22T00:00:00Z
T::11::+2004-11-22T00:00:00Z
T::11::+1984-0

T::11::+2008-10-26T00:00:00Z
T::11::+2005-11-08T00:00:00Z
T::11::+2008-10-26T00:00:00Z
T::11::+2005-11-08T00:00:00Z
T::11::+2008-10-26T00:00:00Z
T::11::+2009-09-01T00:00:00Z
T::11::+2007-07-24T00:00:00Z
T::11::+2009-03-29T00:00:00Z
T::11::+2009-06-16T00:00:00Z
T::11::+2009-12-22T00:00:00Z
T::11::+2009-11-03T00:00:00Z
T::11::+2008-06-22T00:00:00Z
T::11::+1985-08-11T00:00:00Z
T::9::+2014-01-01T00:00:00Z
T::9::+2009-01-01T00:00:00Z
Q4764958
T::9::+1948-01-01T00:00:00Z
T::9::+1948-01-01T00:00:00Z
T::9::+1966-01-01T00:00:00Z
T::11::+1931-10-16T00:00:00Z
T::11::+2011-01-09T00:00:00Z
T::11::+1959-07-02T00:00:00Z
T::11::+1932-06-18T00:00:00Z
T::9::+0523-00-00T00:00:00Z
T::9::+1055-00-00T00:00:00Z
T::9::+1213-00-00T00:00:00Z
T::10::+1939-09-00T00:00:00Z
T::10::+1939-09-00T00:00:00Z
T::11::+1939-09-17T00:00:00Z
T::11::+1939-09-01T00:00:00Z
T::9::+1940-00-00T00:00:00Z
T::11::+1944-10-02T00:00:00Z
T::11::+1918-10-28T00:00:00Z
V::3.7000::http://www.wikidata.org/entity/Q11229
V::4.9000::http://www.w

T::9::+1949-01-01T00:00:00Z
T::11::+1951-11-19T00:00:00Z
T::9::+1960-01-01T00:00:00Z
T::9::+1961-01-01T00:00:00Z
T::11::+1955-01-22T00:00:00Z
T::9::+1958-01-01T00:00:00Z
T::9::+1958-01-01T00:00:00Z
T::9::+1960-01-01T00:00:00Z
T::9::+1961-01-01T00:00:00Z
T::9::+1960-01-01T00:00:00Z
T::9::+1961-01-01T00:00:00Z
T::9::+1961-01-01T00:00:00Z
T::9::+1966-01-01T00:00:00Z
T::9::+1977-01-01T00:00:00Z
T::9::+1977-01-01T00:00:00Z
T::9::+1977-01-01T00:00:00Z
T::11::+1976-12-27T00:00:00Z
T::9::+1984-01-01T00:00:00Z
T::9::+1984-01-01T00:00:00Z
T::9::+1987-00-00T00:00:00Z
T::11::+1995-01-01T00:00:00Z
T::9::+1992-00-00T00:00:00Z
T::9::+1993-00-00T00:00:00Z
T::11::+2009-10-07T00:00:00Z
T::11::+1895-12-21T00:00:00Z
T::11::+1979-11-12T00:00:00Z
T::11::+1923-02-09T00:00:00Z
T::11::+1929-10-22T00:00:00Z
T::11::+1979-11-12T00:00:00Z
T::11::+1895-12-21T00:00:00Z
T::11::+1915-05-09T00:00:00Z
T::11::+1918-04-01T00:00:00Z
T::11::+1923-02-09T00:00:00Z
T::11::+1944-07-02T00:00:00Z
T::11::+1979-11-12T00:00:00Z
T::1

T::11::+1845-03-18T00:00:00Z
T::11::+1863-02-27T00:00:00Z
T::11::+1861-04-11T00:00:00Z
T::9::+1650-00-00T00:00:00Z
T::11::+1913-01-23T00:00:00Z
T::11::+2009-06-27T00:00:00Z
T::9::+1955-00-00T00:00:00Z
T::9::+1946-00-00T00:00:00Z
T::9::+1946-00-00T00:00:00Z
T::9::+1936-01-01T00:00:00Z
T::10::+1931-08-01T00:00:00Z
T::11::+1941-06-22T00:00:00Z
T::9::+1941-01-01T00:00:00Z
T::9::+1941-01-01T00:00:00Z
T::9::+1861-00-00T00:00:00Z
T::9::+1075-00-00T00:00:00Z
T::9::+1070-01-01T00:00:00Z
T::11::+1072-02-07T00:00:00Z
T::9::+1089-00-00T00:00:00Z
T::11::+1086-07-14T00:00:00Z
T::9::+1075-00-00T00:00:00Z
T::11::+1119-03-10T00:00:00Z
T::11::+1911-09-29T00:00:00Z
T::11::+1911-09-29T00:00:00Z
T::11::+1939-09-01T00:00:00Z
T::11::+1931-03-03T00:00:00Z
T::11::+1849-11-13T00:00:00Z
T::11::+1849-11-13T00:00:00Z
T::11::+2015-09-25T00:00:00Z
T::11::+1889-11-08T00:00:00Z
T::9::+1910-00-00T00:00:00Z
T::11::+1951-04-22T00:00:00Z
T::11::+1951-10-19T00:00:00Z
T::11::+1393-07-17T00:00:00Z
T::11::+2014-02-11T00:00:00

#### Dataset stats (lines)
- valid 60
- test 60
- train 600

In [113]:
line

{'title': 'Meridian, Mississippi',
 'tokens': ['@@START@@',
  'Meridian',
  'is',
  'the',
  'sixth',
  'largest',
  'city',
  'in',
  'the',
  'state',
  'of',
  'Mississippi',
  ',',
  'United',
  'States',
  '.',
  '@@END@@',
  'It',
  'is',
  'the',
  'county',
  'seat',
  'of',
  'Lauderdale',
  'County',
  'and',
  'the',
  'principal',
  'city',
  'of',
  'the',
  'Meridian',
  ',',
  'Mississippi',
  'Micropolitan',
  'Statistical',
  'Area',
  '.',
  '@@END@@',
  'Along',
  'major',
  'highways',
  ',',
  'the',
  'city',
  'is',
  '93',
  'mi',
  '(',
  '150',
  'km',
  ')',
  'east',
  'of',
  'Jackson',
  ',',
  'Mississippi',
  ';',
  '@@END@@',
  '154',
  'mi',
  '(',
  '248',
  'km',
  ')',
  'southwest',
  'of',
  'Birmingham',
  ',',
  'Alabama',
  ';',
  '@@END@@',
  '202',
  'mi',
  '(',
  '325',
  'km',
  ')',
  'northeast',
  'of',
  'New',
  'Orleans',
  ',',
  'Louisiana',
  ';',
  'and',
  '231',
  'mi',
  '(',
  '372',
  'km',
  ')',
  'southeast',
  'of',
  'M

In [112]:
annot

{'source': 'KG',
 'id': 'T::11::+1964-06-21T00:00:00Z',
 'relation': ['P570'],
 'parent_id': ['Q2733240'],
 'span': [669, 670]}

In [109]:
import time

for ix in range(len(embed_list)):
    print(embed_list[ix])
    time.sleep(1)
    clear_output()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


KeyboardInterrupt: 

In [128]:
npfile = np.load("valid.npz")
npfile

In [129]:
npfile.files

['arr_0',
 'arr_1',
 'arr_2',
 'arr_3',
 'arr_4',
 'arr_5',
 'arr_6',
 'arr_7',
 'arr_8',
 'arr_9',
 'arr_10',
 'arr_11',
 'arr_12',
 'arr_13',
 'arr_14',
 'arr_15',
 'arr_16',
 'arr_17',
 'arr_18',
 'arr_19',
 'arr_20',
 'arr_21',
 'arr_22',
 'arr_23',
 'arr_24',
 'arr_25',
 'arr_26',
 'arr_27',
 'arr_28',
 'arr_29',
 'arr_30',
 'arr_31',
 'arr_32',
 'arr_33',
 'arr_34',
 'arr_35',
 'arr_36',
 'arr_37',
 'arr_38',
 'arr_39',
 'arr_40',
 'arr_41',
 'arr_42',
 'arr_43',
 'arr_44',
 'arr_45',
 'arr_46',
 'arr_47',
 'arr_48',
 'arr_49',
 'arr_50',
 'arr_51',
 'arr_52',
 'arr_53',
 'arr_54',
 'arr_55',
 'arr_56',
 'arr_57',
 'arr_58',
 'arr_59']

In [127]:
npfile['arr_0']

array([[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [-0.5412,  0.4502,  0.2182, ..., -0.099 , -0.1066,  0.134 ],
       [-0.5412,  0.4502,  0.2182, ..., -0.099 , -0.1066,  0.134 ],
       ...,
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]])

In [98]:
line['annotations']

[{'source': 'WIKI',
  'id': 'Q822935',
  'relation': ['@@NEW@@'],
  'parent_id': ['Q822935'],
  'span': [1, 3]},
 {'source': 'WIKI',
  'id': 'Q1788582',
  'relation': ['@@NEW@@'],
  'parent_id': ['Q1788582'],
  'span': [5, 7]},
 {'source': 'WIKI',
  'id': 'Q35657',
  'relation': ['@@NEW@@'],
  'parent_id': ['Q35657'],
  'span': [13, 15]},
 {'source': 'WIKI',
  'id': 'Q1408',
  'relation': ['R:P31', 'P131'],
  'parent_id': ['Q35657', 'Q822935'],
  'span': [16, 18]},
 {'source': 'COREF',
  'id': 'Q822935',
  'relation': ['@@REFLEXIVE@@', 'R:P131'],
  'parent_id': ['Q822935', 'Q1408'],
  'span': [20, 21]},
 {'source': 'WIKI',
  'id': 'Q811383',
  'relation': ['R:P131'],
  'parent_id': ['Q1408'],
  'span': [32, 35]},
 {'source': 'WIKI',
  'id': 'Q811493',
  'relation': ['@@NEW@@'],
  'parent_id': ['Q811493'],
  'span': [41, 44]},
 {'source': 'WIKI',
  'id': 'Q1073974',
  'relation': ['@@NEW@@'],
  'parent_id': ['Q1073974'],
  'span': [45, 47]},
 {'source': 'WIKI',
  'id': 'Q497795',
  'rel

In [99]:
' '.join(line['tokens'])

'@@START@@ Route 50 is a state highway in the southern part of the U.S. state of New Jersey . @@END@@ It runs 26.02 mi ( 41.88 km ) from an intersection with U.S. Route 9 ( US 9 ) and the Garden State Parkway in Upper Township , Cape May County to an intersection with US 30 and County Route 563 ( CR 563 ) in Egg Harbor City , Atlantic County . @@END@@ The route , which is mostly a two - lane undivided road , passes through mostly rural areas of Atlantic and Cape May counties as well as the communities of Tuckahoe , Corbin City , Estell Manor , and Mays Landing . @@END@@ NJ 50 intersects several roads , including Route 49 in Tuckahoe , US 40 in Mays Landing , and US 322 and the Atlantic City Expressway in Hamilton Township . @@END@@ The portion of current Route 50 between Seaville and Petersburg received funding in 1910 to become a spur of the Ocean Highway . @@END@@ In 1917 , what is now Route 50 was designated a part of pre-1927 Route 14 , a route that was to run from Cape May to Egg 

In [51]:
# with open('test.npy', 'wb') as f:
#     np.save(f, np.array([1, 2]))
#     np.save(f, np.array([1, 3]))
# with open('test.npy', 'rb') as f:
#     a = np.load(f)
#     b = np.load(f)
# a, b # [1 2] [1 3]

(array([1, 2]), array([1, 3]))

In [61]:
# with open('test2.npy', 'ab') as f:
#     np.save(f, np.array([6, 2]))

# with open('test2.npy', 'ab') as f:
#     np.save(f, np.array([1, 3]))
    
# with open('test2.npy', 'ab') as f:
#     np.save(f, np.array([2, 3]))
    
# with open('test2.npy', 'rb') as f:
#     x = np.load(f)
#     y = np.load(f)
#     z = np.load(f)
# x, y, z # [1 2] [1 3]

(array([1, 2]), array([1, 3]), array([2, 3]))

## Create PyTorch Dataset

In [7]:
embeds_wktxt = pd.read_csv("embeds_wktxt.csv")
embeds_wktxt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46685 entries, 0 to 46684
Columns: 201 entries, id to embedding_200
dtypes: float64(200), object(1)
memory usage: 71.6+ MB


In [8]:
from datasets import load_dataset

data_files = {"train": train, "valid": valid, "test": test}
wikitest2_dataset = load_dataset("json", data_files=data_files)
wikitest2_dataset

Using custom data configuration default-c5571b3a8bc0c3d4
Found cached dataset json (/home/a2khatun/.cache/huggingface/datasets/json/default-c5571b3a8bc0c3d4/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'tokens', 'annotations'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['title', 'tokens', 'annotations'],
        num_rows: 60
    })
    test: Dataset({
        features: ['title', 'tokens', 'annotations'],
        num_rows: 60
    })
})

In [9]:
# It provides a Process class that allows us to check the memory usage of the current process as follows:

import psutil

# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used: 489.52 MB


In [10]:
def get_kg_embedding(data):
    tokens = data["tokens"]
    annotations = data['annotations']
    
    embed_list = np.zeros((len(tokens), 200))
            
    for annot in annotations:
        start, end = annot['span']
        qid = annot['id']
        df = embeds_wktxt[embeds_wktxt['id']==qid]
        if len(df)>0:
            embed_list[start:end] = np.tile(df.iloc[0,1:].values.reshape((1,200)),(end-start, 1))
                    
    return {"kg_embedding": embed_list}

def get_kg_embedding_batched(data):
    """
    When you specify batched=True the function receives a dictionary with the fields of the dataset, 
    but each value is now a list of values, and not just a single value. 
    """
    tokens_list = data["tokens"]
    annotations_list = data['annotations']
    ln = len(tokens_list)
    embed_list = []
    
    for i in range(ln):
        tokens = tokens_list[i]
        annotations = annotations_list[i]
        embeds = np.zeros((len(tokens), 200))
        
        for annot in annotations:
            start, end = annot['span']
            qid = annot['id']
            df = embeds_wktxt[embeds_wktxt['id']==qid]
            if len(df)>0:
                embeds[start:end] = np.tile(df.iloc[0,1:].values.reshape((1,200)),(end-start, 1))
        
        embed_list.append(embeds)

    return {"kg_embedding": embed_list}

kg_wikitest2_dataset = wikitest2_dataset.map(get_kg_embedding_batched, 
                                             batched=True, 
                                             batch_size=100, 
                                             keep_in_memory=False)
kg_wikitest2_dataset

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 60
    })
})

In [20]:
# It provides a Process class that allows us to check the memory usage of the current process as follows:

import psutil

# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

# print(f"Number of files in dataset : {wikitest2_dataset.dataset_size}")
# size_gb = wikitest2_dataset.dataset_size / (1024**3)
# print(f"Dataset size (cache file) : {size_gb:.2f} GB")

RAM used: 5187.64 MB


In [ ]:
kg_wikitest2_dataset_file = "kg_wikitest2_dataset_file"

kg_wikitest2_dataset.save_to_disk(kg_wikitest2_dataset_file)

In [13]:
kg_wikitest2_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 60
    })
})

In [58]:
kg_wikitest2_dataset_loaded = load_from_disk(kg_wikitest2_dataset_file)

In [57]:
kg_wikitest2_dataset_loaded

DatasetDict({
    train: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 60
    })
})

In [19]:
del kg_wikitest2_dataset_loaded

In [17]:
kg_wikitest2_dataset.map()

DatasetDict({
    train: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 60
    })
})

In [23]:
for row in kg_wikitest2_dataset["valid"]["tokens"]:
    print(len(row))

1352
10663
2474
10767
3619
1247
4262
1559
3602
1689
2038
1456
6072
1832
3524
18805
3843
1291
1535
1955
2173
5384
3375
875
2588
857
3129
2023
6652
6681
5720
3951
5427
2929
5950
1301
4328
4530
2648
2046
4824
1364
3407
4253
1325
1287
2078
1705
5747
1314
5267
2746
7504
2746
1887
5184
2060
772
1358
3096


In [24]:
kg_wikitest2_dataset["valid"]["tokens"][0]

['@@START@@',
 'Route',
 '50',
 'is',
 'a',
 'state',
 'highway',
 'in',
 'the',
 'southern',
 'part',
 'of',
 'the',
 'U.S.',
 'state',
 'of',
 'New',
 'Jersey',
 '.',
 '@@END@@',
 'It',
 'runs',
 '26.02',
 'mi',
 '(',
 '41.88',
 'km',
 ')',
 'from',
 'an',
 'intersection',
 'with',
 'U.S.',
 'Route',
 '9',
 '(',
 'US',
 '9',
 ')',
 'and',
 'the',
 'Garden',
 'State',
 'Parkway',
 'in',
 'Upper',
 'Township',
 ',',
 'Cape',
 'May',
 'County',
 'to',
 'an',
 'intersection',
 'with',
 'US',
 '30',
 'and',
 'County',
 'Route',
 '563',
 '(',
 'CR',
 '563',
 ')',
 'in',
 'Egg',
 'Harbor',
 'City',
 ',',
 'Atlantic',
 'County',
 '.',
 '@@END@@',
 'The',
 'route',
 ',',
 'which',
 'is',
 'mostly',
 'a',
 'two',
 '-',
 'lane',
 'undivided',
 'road',
 ',',
 'passes',
 'through',
 'mostly',
 'rural',
 'areas',
 'of',
 'Atlantic',
 'and',
 'Cape',
 'May',
 'counties',
 'as',
 'well',
 'as',
 'the',
 'communities',
 'of',
 'Tuckahoe',
 ',',
 'Corbin',
 'City',
 ',',
 'Estell',
 'Manor',
 ',',
 'a

In [38]:
ids = [[i for i,token in enumerate(tokens) if token!="@@START@@" and token!="@@END@@"]
                  for tokens in kg_wikitest2_dataset["valid"]["tokens"][:2]]

In [37]:
len(ids[0])

1299

In [44]:
kg_wikitest2_dataset["valid"]["tokens"][0][ids[0][0]]

'Route'

In [46]:
[kg_wikitest2_dataset["valid"]["tokens"][0][i] for i in ids[0][:100]]

['Route',
 '50',
 'is',
 'a',
 'state',
 'highway',
 'in',
 'the',
 'southern',
 'part',
 'of',
 'the',
 'U.S.',
 'state',
 'of',
 'New',
 'Jersey',
 '.',
 'It',
 'runs',
 '26.02',
 'mi',
 '(',
 '41.88',
 'km',
 ')',
 'from',
 'an',
 'intersection',
 'with',
 'U.S.',
 'Route',
 '9',
 '(',
 'US',
 '9',
 ')',
 'and',
 'the',
 'Garden',
 'State',
 'Parkway',
 'in',
 'Upper',
 'Township',
 ',',
 'Cape',
 'May',
 'County',
 'to',
 'an',
 'intersection',
 'with',
 'US',
 '30',
 'and',
 'County',
 'Route',
 '563',
 '(',
 'CR',
 '563',
 ')',
 'in',
 'Egg',
 'Harbor',
 'City',
 ',',
 'Atlantic',
 'County',
 '.',
 'The',
 'route',
 ',',
 'which',
 'is',
 'mostly',
 'a',
 'two',
 '-',
 'lane',
 'undivided',
 'road',
 ',',
 'passes',
 'through',
 'mostly',
 'rural',
 'areas',
 'of',
 'Atlantic',
 'and',
 'Cape',
 'May',
 'counties',
 'as',
 'well',
 'as',
 'the',
 'communities']

In [76]:
def filter_text_batched(data):
    tokens_list = data["tokens"]
    kg_embeds_list = data['kg_embedding']
    indices_list = [[i for i,token in enumerate(tokens) 
                         if token!="@@START@@" and token!="@@END@@"]
                            for tokens in tokens_list]
    new_tokens_list = []
    new_kg_embedding_list = []
    
    for indices, tokens, kg_embeds in zip(indices_list, tokens_list, kg_embeds_list):
        
        new_tokens = [tokens[ind] for ind in indices]
        new_kg_embeds = [kg_embeds[ind] for ind in indices]
        
        new_tokens_list.append(new_tokens)
        new_kg_embedding_list.append(new_kg_embeds)
        
    
    return {"tokens": new_tokens_list, "kg_embedding": new_kg_embedding_list}

kg_wikitest2_dataset_filtered = kg_wikitest2_dataset.map(filter_text_batched, 
                                             batched=True, 
                                             batch_size=100, 
                                             keep_in_memory=False)\
                                    .remove_columns(["title", "annotations"])
kg_wikitest2_dataset_filtered

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['tokens', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['tokens', 'kg_embedding'],
        num_rows: 60
    })
})

In [78]:
for row, row_filtered in zip(kg_wikitest2_dataset["valid"]["tokens"], 
                             kg_wikitest2_dataset_filtered["valid"]["tokens"]):
#     print(len(row),len(row_filtered))
    print(len(row)-len(row_filtered))

53
438
98
413
134
56
156
57
153
72
71
54
220
72
153
650
184
42
57
75
93
206
126
39
105
48
112
80
244
265
197
136
183
126
241
57
176
152
111
88
238
56
140
130
44
58
67
62
212
43
199
86
205
106
69
225
90
32
59
100


In [80]:
kg_wikitest2_dataset_filtered_file = "kg_wikitest2_dataset_filtered_file"

kg_wikitest2_dataset_filtered.save_to_disk(kg_wikitest2_dataset_filtered_file)

In [81]:
kg_wikitest2_dataset_filtered

DatasetDict({
    train: Dataset({
        features: ['tokens', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['tokens', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['tokens', 'kg_embedding'],
        num_rows: 60
    })
})

In [70]:
kg_wikitest2_dataset_filtered_loaded = load_from_disk(kg_wikitest2_dataset_filtered_file)

In [71]:
kg_wikitest2_dataset_filtered_loaded

DatasetDict({
    train: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['title', 'tokens', 'annotations', 'kg_embedding'],
        num_rows: 60
    })
})

In [72]:
for row, row_filtered in zip(kg_wikitest2_dataset["valid"]["tokens"], 
                             kg_wikitest2_dataset_filtered_loaded["valid"]["tokens"]):
    print(len(row)-len(row_filtered))

53
438
98
413
134
56
156
57
153
72
71
54
220
72
153
650
184
42
57
75
93
206
126
39
105
48
112
80
244
265
197
136
183
126
241
57
176
152
111
88
238
56
140
130
44
58
67
62
212
43
199
86
205
106
69
225
90
32
59
100


## Prepare dataset for input to model

In [6]:
kg_wikitest2_dataset_filtered_file = "kg_wikitest2_dataset_filtered_file"

kg_wikitest2_dataset_filtered = load_from_disk(kg_wikitest2_dataset_filtered_file)

In [7]:
kg_wikitest2_dataset_filtered

DatasetDict({
    train: Dataset({
        features: ['tokens', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['tokens', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['tokens', 'kg_embedding'],
        num_rows: 60
    })
})

In [83]:
from datasets import load_dataset
eg_dataset = load_dataset("zhengxuanzenwu/wikitext-2-split-128", split="test")
eg_dataset

Using custom data configuration zhengxuanzenwu--wikitext-2-split-128-f504347a654a9463
Found cached dataset parquet (/home/a2khatun/.cache/huggingface/datasets/zhengxuanzenwu___parquet/zhengxuanzenwu--wikitext-2-split-128-f504347a654a9463/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['text'],
    num_rows: 8192
})

In [85]:
eg_dataset["text"][0]

'Most of the North Korean units began conducting delaying actions attempting to get as much of their army as possible into North Korea .'

In [10]:
bert_model_name = "distilbert-base-uncased" ##"bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

In [19]:
tokenizer.do_lower_case

True

In [22]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [23]:
tokenizer.do_lower_case

False

In [26]:
sml = kg_wikitest2_dataset_filtered["valid"].select(range(5))
sml

Dataset({
    features: ['tokens', 'kg_embedding'],
    num_rows: 5
})

In [31]:
for tkns in sml["tokens"]:
    print(len(tkns))
    ## whitespace tokenize
    re_tkns = (" ".join(tkns)).strip().split()
    print(len(re_tkns), end="\n\n")

1299
1299

10225
10225

2376
2376

10354
10354

3485
3485



In [29]:
" ".join(sml["tokens"][0])

'Route 50 is a state highway in the southern part of the U.S. state of New Jersey . It runs 26.02 mi ( 41.88 km ) from an intersection with U.S. Route 9 ( US 9 ) and the Garden State Parkway in Upper Township , Cape May County to an intersection with US 30 and County Route 563 ( CR 563 ) in Egg Harbor City , Atlantic County . The route , which is mostly a two - lane undivided road , passes through mostly rural areas of Atlantic and Cape May counties as well as the communities of Tuckahoe , Corbin City , Estell Manor , and Mays Landing . NJ 50 intersects several roads , including Route 49 in Tuckahoe , US 40 in Mays Landing , and US 322 and the Atlantic City Expressway in Hamilton Township . The portion of current Route 50 between Seaville and Petersburg received funding in 1910 to become a spur of the Ocean Highway . In 1917 , what is now Route 50 was designated a part of pre-1927 Route 14 , a route that was to run from Cape May to Egg Harbor City . Route 50 was designated onto its cur

In [34]:
def tokenize_function(examples):
    result = tokenizer([" ".join(eg) for eg in examples["tokens"]])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
# tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# tokenized_datasets

tokenized_sml = sml.map(tokenize_function, batched=True)
tokenized_sml

  0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1432 > 512). Running this sequence through the model will result in indexing errors


Dataset({
    features: ['tokens', 'kg_embedding', 'input_ids', 'attention_mask', 'word_ids'],
    num_rows: 5
})

In [37]:
for tkns in tokenized_sml["word_ids"]:
    print(tkns[-5:])

[1318, 1319, 1320, 1321, None]
[10507, 10507, 10508, 10509, None]
[2454, 2455, 2456, 2457, None]
[10756, 10757, 10758, 10759, None]
[3537, 3538, 3539, 3540, None]


In [43]:
sml["tokens"][0]

['Route',
 '50',
 'is',
 'a',
 'state',
 'highway',
 'in',
 'the',
 'southern',
 'part',
 'of',
 'the',
 'U.S.',
 'state',
 'of',
 'New',
 'Jersey',
 '.',
 'It',
 'runs',
 '26.02',
 'mi',
 '(',
 '41.88',
 'km',
 ')',
 'from',
 'an',
 'intersection',
 'with',
 'U.S.',
 'Route',
 '9',
 '(',
 'US',
 '9',
 ')',
 'and',
 'the',
 'Garden',
 'State',
 'Parkway',
 'in',
 'Upper',
 'Township',
 ',',
 'Cape',
 'May',
 'County',
 'to',
 'an',
 'intersection',
 'with',
 'US',
 '30',
 'and',
 'County',
 'Route',
 '563',
 '(',
 'CR',
 '563',
 ')',
 'in',
 'Egg',
 'Harbor',
 'City',
 ',',
 'Atlantic',
 'County',
 '.',
 'The',
 'route',
 ',',
 'which',
 'is',
 'mostly',
 'a',
 'two',
 '-',
 'lane',
 'undivided',
 'road',
 ',',
 'passes',
 'through',
 'mostly',
 'rural',
 'areas',
 'of',
 'Atlantic',
 'and',
 'Cape',
 'May',
 'counties',
 'as',
 'well',
 'as',
 'the',
 'communities',
 'of',
 'Tuckahoe',
 ',',
 'Corbin',
 'City',
 ',',
 'Estell',
 'Manor',
 ',',
 'and',
 'Mays',
 'Landing',
 '.',
 'NJ',

In [29]:
" ".join(sml["tokens"][0])

'Route 50 is a state highway in the southern part of the U.S. state of New Jersey . It runs 26.02 mi ( 41.88 km ) from an intersection with U.S. Route 9 ( US 9 ) and the Garden State Parkway in Upper Township , Cape May County to an intersection with US 30 and County Route 563 ( CR 563 ) in Egg Harbor City , Atlantic County . The route , which is mostly a two - lane undivided road , passes through mostly rural areas of Atlantic and Cape May counties as well as the communities of Tuckahoe , Corbin City , Estell Manor , and Mays Landing . NJ 50 intersects several roads , including Route 49 in Tuckahoe , US 40 in Mays Landing , and US 322 and the Atlantic City Expressway in Hamilton Township . The portion of current Route 50 between Seaville and Petersburg received funding in 1910 to become a spur of the Ocean Highway . In 1917 , what is now Route 50 was designated a part of pre-1927 Route 14 , a route that was to run from Cape May to Egg Harbor City . Route 50 was designated onto its cur

In [40]:
decoded_text = tokenizer.decode(tokenized_sml["input_ids"][0])

len(decoded_text.split()) ## cuz the output removes space before . and , etc

1202

In [41]:
decoded_text

'[CLS] Route 50 is a state highway in the southern part of the U. S. state of New Jersey. It runs 26. 02 mi ( 41. 88 km ) from an intersection with U. S. Route 9 ( US 9 ) and the Garden State Parkway in Upper Township, Cape May County to an intersection with US 30 and County Route 563 ( CR 563 ) in Egg Harbor City, Atlantic County. The route, which is mostly a two - lane undivided road, passes through mostly rural areas of Atlantic and Cape May counties as well as the communities of Tuckahoe, Corbin City, Estell Manor, and Mays Landing. NJ 50 intersects several roads, including Route 49 in Tuckahoe, US 40 in Mays Landing, and US 322 and the Atlantic City Expressway in Hamilton Township. The portion of current Route 50 between Seaville and Petersburg received funding in 1910 to become a spur of the Ocean Highway. In 1917, what is now Route 50 was designated a part of pre - 1927 Route 14, a route that was to run from Cape May to Egg Harbor City. Route 50 was designated onto its current a

In [42]:
manually_decoded = []

for tkn in tokenized_sml["input_ids"][0]:
    manually_decoded.append(tokenizer.decode([tkn]))
    
manually_decoded

['[CLS]',
 'Route',
 '50',
 'is',
 'a',
 'state',
 'highway',
 'in',
 'the',
 'southern',
 'part',
 'of',
 'the',
 'U',
 '.',
 'S',
 '.',
 'state',
 'of',
 'New',
 'Jersey',
 '.',
 'It',
 'runs',
 '26',
 '.',
 '02',
 'mi',
 '(',
 '41',
 '.',
 '88',
 'km',
 ')',
 'from',
 'an',
 'intersection',
 'with',
 'U',
 '.',
 'S',
 '.',
 'Route',
 '9',
 '(',
 'US',
 '9',
 ')',
 'and',
 'the',
 'Garden',
 'State',
 'Parkway',
 'in',
 'Upper',
 'Township',
 ',',
 'Cape',
 'May',
 'County',
 'to',
 'an',
 'intersection',
 'with',
 'US',
 '30',
 'and',
 'County',
 'Route',
 '56',
 '##3',
 '(',
 'CR',
 '56',
 '##3',
 ')',
 'in',
 'Egg',
 'Harbor',
 'City',
 ',',
 'Atlantic',
 'County',
 '.',
 'The',
 'route',
 ',',
 'which',
 'is',
 'mostly',
 'a',
 'two',
 '-',
 'lane',
 'und',
 '##iv',
 '##ided',
 'road',
 ',',
 'passes',
 'through',
 'mostly',
 'rural',
 'areas',
 'of',
 'Atlantic',
 'and',
 'Cape',
 'May',
 'counties',
 'as',
 'well',
 'as',
 'the',
 'communities',
 'of',
 'Tu',
 '##ck',
 '##ah',


In [47]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [181]:
from transformers import BertTokenizer, DistilBertTokenizer

class BertTokenizerModified(DistilBertTokenizer): #BertTokenizer
    def __init__(self,vocab_file,**kwargs):
        
        super().__init__(vocab_file, **kwargs)
    
        self.tokenized_list = []

    def _tokenize(self, text):
        token_list = text.split()
        split_tokens = []
        tokenized_list = []
        
        if self.do_basic_tokenize:
            for token in token_list:

                # If the token is part of the never_split set
                if token in self.basic_tokenizer.never_split:
                    split_tokens.append(token)
                    tokenized_list.append(1)
                else:
                    word_tokenized = self.wordpiece_tokenizer.tokenize(token)
                    split_tokens += word_tokenized
                    tokenized_list.append(len(word_tokenized))
                
#         else:
            ## donno
        self.tokenized_list.append(tokenized_list)
        return split_tokens

In [182]:
my_tokenizer = BertTokenizerModified.from_pretrained("distilbert-base-cased")
my_tokenizer

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerModified'.


PreTrainedTokenizer(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [183]:
my_tokenizer.is_fast

False

In [184]:
tokenized = my_tokenizer(["You're a good boy", "Aww, so are you!"])
tokenized

{'input_ids': [[101, 1192, 28131, 1874, 170, 1363, 2298, 102], [101, 138, 2246, 2246, 28136, 1177, 1132, 1128, 28125, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [185]:
my_tokenizer.tokenized_list

[[3, 1, 1, 1], [4, 1, 1, 2]]

In [167]:
print(my_tokenizer.decode(tokenized["input_ids"][0]))

[CLS] You're a good boy [SEP]


In [91]:
%debug

> /home/a2khatun/.venv/analysis/lib/python3.10/site-packages/transformers/tokenization_utils.py(731)_batch_encode_plus()
    729                 ids, pair_ids = ids_or_pair_ids, None
    730             else:
--> 731                 ids, pair_ids = ids_or_pair_ids
    732 
    733             first_ids = get_input_ids(ids)

ipdb> p ids_or_pair_ids
['This', 'is', 'an', 'example']
ipdb> q


In [178]:
sml

Dataset({
    features: ['tokens', 'kg_embedding'],
    num_rows: 5
})

In [202]:
x = [1,2,3,5,6]
y = [0] + x
[y[i-1]+y[i] for i in range(1,len(y))]

[1, 3, 5, 8, 11]

In [198]:
def my_tokenize_function(examples):
    
    ## tokenize
    my_tokenizer.tokenized_list = []
    result = my_tokenizer([" ".join(eg) for eg in examples["tokens"]])
    if my_tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    
    ## save word to token mapping
    ## 3, 1, 1 means the first word got divided into 3 tokens, the next into 1, and the next into 1 again
    result["word_tokens"] = my_tokenizer.tokenized_list
    
    return result


tokenized_sml = sml.map(my_tokenize_function, batched=True)
tokenized_sml

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['tokens', 'kg_embedding', 'input_ids', 'attention_mask', 'word_tokens'],
    num_rows: 5
})

In [203]:
len(tokenized_sml)

5

## KG embed and tokenize

In [223]:
embeds_wktxt = pd.read_csv("embeds_wktxt.csv")
embeds_wktxt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46685 entries, 0 to 46684
Columns: 201 entries, id to embedding_200
dtypes: float64(200), object(1)
memory usage: 71.6+ MB


In [267]:
from datasets import load_dataset

data_files = {"train": train, "valid": valid, "test": test}
wikitest2_dataset = load_dataset("json", data_files=data_files)
wikitest2_dataset

Using custom data configuration default-c5571b3a8bc0c3d4
Found cached dataset json (/home/a2khatun/.cache/huggingface/datasets/json/default-c5571b3a8bc0c3d4/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'tokens', 'annotations'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['title', 'tokens', 'annotations'],
        num_rows: 60
    })
    test: Dataset({
        features: ['title', 'tokens', 'annotations'],
        num_rows: 60
    })
})

In [214]:
from transformers import BertTokenizer, DistilBertTokenizer

class BertTokenizerModified(DistilBertTokenizer): #BertTokenizer
    def __init__(self,vocab_file,**kwargs):
        
        super().__init__(vocab_file, never_split=["@@START@@", "@@END@@"], **kwargs)
    
        self.tokenized_list = []

    def _tokenize(self, text):
        token_list = text.split()
        split_tokens = []
        tokenized_list = []
        
        if self.do_basic_tokenize:
            for token in token_list:

                # If the token is part of the never_split set
                if token in self.basic_tokenizer.never_split:
                    split_tokens.append(token)
                    tokenized_list.append(1)
                else:
                    word_tokenized = self.wordpiece_tokenizer.tokenize(token)
                    split_tokens += word_tokenized
                    tokenized_list.append(len(word_tokenized))
                
#         else:
            ## donno
        self.tokenized_list.append(tokenized_list)
        return split_tokens

In [215]:
my_tokenizer = BertTokenizerModified.from_pretrained("distilbert-base-cased")
my_tokenizer

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerModified'.


PreTrainedTokenizer(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [216]:
tokenized = my_tokenizer(["@@START@@ You're a good boy @@END@@", "Aww, so are you!"])
tokenized

{'input_ids': [[101, 100, 1192, 28131, 1874, 170, 1363, 2298, 100, 102], [101, 138, 2246, 2246, 28136, 1177, 1132, 1128, 28125, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [217]:
my_tokenizer.tokenized_list

[[1, 3, 1, 1, 1, 1], [4, 1, 1, 2]]

In [218]:
print(my_tokenizer.decode(tokenized["input_ids"][0]))

for tk in tokenized["input_ids"][0]:
    print(my_tokenizer.decode(tk))

[CLS] [UNK] You're a good boy [UNK] [SEP]
[ C L S ]
[ U N K ]
Y o u
# # '
# # r e
a
g o o d
b o y
[ U N K ]
[ S E P ]


In [230]:
sml = wikitest2_dataset["train"].select(range(5))
sml

Dataset({
    features: ['title', 'tokens', 'annotations'],
    num_rows: 5
})

In [231]:
def get_cumm(vals):
    cumm = 0
    res = [0] ## len of res is 1 more than vals, with an initial 0
    for val in vals:
        cumm += val
        res.append(cumm)
    return res

def my_tokenize_function(data):
    
    ## tokenize
    my_tokenizer.tokenized_list = []
    result = my_tokenizer([" ".join(eg) for eg in data["tokens"]])
    if my_tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    
    ## save word to token mapping
    ## 3, 1, 1 means the first word got divided into 3 tokens, the next into 1, and the next into 1 again
    result["word_tokens"] = my_tokenizer.tokenized_list
    result["cummulative_word_tokens"] = [get_cumm(x) for x in result["word_tokens"]]
    return result


tokenized_sml = sml.map(my_tokenize_function, batched=True)
tokenized_sml

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens'],
    num_rows: 5
})

- `tokens` are the given list of tokens from wikitext2
- `input_ids` are what come from tokenization, they divide certain words into multiple pieces, and each sentence has a CLS and a SEP
- `word_tokens` is the length of `tokens`. For each token in `token`, it mentions how many sub-words it was divided into due to word piece tokenization
- `cummulative_word_tokens` is a cummulative sum of `word_tokens`, with an extra 0 in the beginning

##### Process
index of a token in `token` can be found in `input_ids` by `cummulative_word_tokens`. if `ix` is the index of a word in `token`, its beginning index in `input_ids` is `cummulative_word_tokens[ix] + 1`, the +1 is because `input_ids` has a CLS in the beginning. `token[ix]` spans from `input_ids[cummulative_word_tokens[ix] + 1]` to `input_ids[cummulative_word_tokens[ix+1] + 1]`

In [258]:
def get_kg_embedding_batched(data):
    """
    When you specify batched=True the function receives a dictionary with the fields of the dataset, 
    but each value is now a list of values, and not just a single value. 
    """
    input_ids_list = data["input_ids"]
    annotations_list = data['annotations']
    cummulative_word_tokens_list = data["cummulative_word_tokens"]
    
    batch_size = len(input_ids_list)
    embed_list = [] ## len will be batch_size
    
    for i in range(batch_size):
        input_ids = input_ids_list[i]
        annotations = annotations_list[i]
        embeds = np.zeros((len(input_ids), 200)) ## CLS, SEP will have np.zeros, like unknown words
        
        for annot in annotations:
            start_ix, end_ix = annot['span']
            start = cummulative_word_tokens_list[i][start_ix] + 1
            end = cummulative_word_tokens_list[i][end_ix] + 1
            
            qid = annot['id']
            df = embeds_wktxt[embeds_wktxt['id']==qid]
            if len(df)>0:
                embeds[start:end] = np.tile(df.iloc[0,1:].values.reshape((1,200)),(end-start, 1))
        
        embed_list.append(embeds)

    return {"kg_embedding": embed_list}

In [259]:
kg_sml_dataset = tokenized_sml.map(get_kg_embedding_batched, 
                                             batched=True, 
                                             batch_size=100, 
                                             keep_in_memory=False)
kg_sml_dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens', 'kg_embedding'],
    num_rows: 5
})

In [260]:
# for elem in kg_sml_dataset:
#     for k,v in elem.items():
#         print(k, len(v))
#     print()

In [261]:
for elem in kg_sml_dataset:
    print(elem["tokens"][:100])
    break

['@@START@@', 'Allah', '(', '/ˈælə', ',', 'ˈɑːlə', ',', 'əlˈlɑː/', ';', '@@END@@', 'Arabic', ':', 'الله', '\u200e', ',', 'translit', '.', '@@END@@', 'Allāh', ',', 'IPA', ':', '[', 'ɑɫˈɫɑː(h', ')', ']', '(', 'listen', ')', ')', 'is', 'the', 'Arabic', 'word', 'for', 'God', 'in', 'Abrahamic', 'religions', '.', '@@END@@', 'In', 'the', 'English', 'language', ',', 'the', 'word', 'generally', 'refers', 'to', 'God', 'in', 'Islam', '.', '@@END@@', 'The', 'word', 'is', 'thought', 'to', 'be', 'derived', 'by', 'contraction', 'from', 'al', '-', 'ilāh', ',', 'which', 'means', '"', 'the', 'god', '"', ',', 'and', 'is', 'related', 'to', 'El', 'and', 'Elah', ',', 'the', 'Hebrew', 'and', 'Aramaic', 'words', 'for', 'God', '.', '@@END@@', 'The', 'word', 'Allah', 'has', 'been', 'used']


In [262]:
for elem in kg_sml_dataset:
    print(" ".join(elem["tokens"]))

@@START@@ Allah ( /ˈælə , ˈɑːlə , əlˈlɑː/ ; @@END@@ Arabic : الله ‎ , translit . @@END@@ Allāh , IPA : [ ɑɫˈɫɑː(h ) ] ( listen ) ) is the Arabic word for God in Abrahamic religions . @@END@@ In the English language , the word generally refers to God in Islam . @@END@@ The word is thought to be derived by contraction from al - ilāh , which means " the god " , and is related to El and Elah , the Hebrew and Aramaic words for God . @@END@@ The word Allah has been used by Arabic people of different religions since pre - Islamic times . @@END@@ More specifically , it has been used as a term for God by Muslims ( both Arab and non - Arab ) and Arab Christians . @@END@@ It is also often , albeit not exclusively , used in this way by Bábists , Bahá'ís , Mandaeans , Indonesian and Maltese Christians , and Mizrahi Jews . @@END@@ Similar usage by Christians and Sikhs in West Malaysia has recently led to political and legal controversies . @@END@@ The etymology of the word Allāh has been discussed e

@@START@@ In cell biology , the nucleus ( pl . nuclei ; from Latin nucleus or nuculeus , meaning kernel or seed ) is a membrane -enclosed organelle found in eukaryotic cells . @@END@@ Eukaryotes usually have a single nucleus , but a few cell types , such as mammalian red blood cells , have no nuclei , and a few others including osteoclasts have many . @@END@@ Cell nuclei contain most of the cell 's genetic material , organized as multiple long linear DNA molecules in a complex with a large variety of proteins , such as histones , to form chromosomes . @@END@@ The genes within these chromosomes are the cell 's nuclear genome and are structured in such a way to promote cell function . @@END@@ The nucleus maintains the integrity of genes and controls the activities of the cell by regulating gene expression —the nucleus is , therefore , the control center of the cell . @@END@@ The main structures making up the nucleus are the nuclear envelope , a double membrane that encloses the entire or

In [265]:
for elem in kg_sml_dataset:
    print(elem["word_tokens"][:100])
    break

[1, 1, 1, 5, 1, 5, 1, 7, 1, 1, 1, 1, 4, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [266]:
for elem in kg_sml_dataset:
    for mp in elem["annotations"][:100]:
        print(mp["span"])
    break

[1, 2]
[3, 8]
[10, 11]
[15, 18]
[22, 26]
[32, 33]
[35, 39]
[51, 54]
[64, 65]
[66, 67]
[68, 69]
[74, 75]
[81, 82]
[83, 84]
[86, 87]
[88, 89]
[91, 92]
[95, 96]
[96, 97]
[101, 103]
[107, 110]
[124, 125]
[126, 127]
[129, 130]
[133, 134]
[136, 138]
[154, 155]
[156, 157]
[158, 159]
[160, 161]
[162, 163]
[166, 168]
[175, 176]
[190, 191]
[255, 256]
[268, 269]
[271, 272]
[274, 275]
[281, 282]
[283, 284]
[292, 293]
[322, 323]
[332, 334]
[336, 337]
[338, 339]
[343, 344]
[371, 373]
[379, 380]
[384, 386]
[395, 397]
[407, 408]
[434, 435]
[456, 457]
[461, 463]
[480, 482]
[488, 489]
[499, 501]
[512, 514]
[515, 516]
[525, 526]
[535, 536]
[541, 542]
[556, 557]
[558, 559]
[571, 572]
[578, 579]
[580, 581]
[587, 588]
[592, 593]
[601, 602]
[612, 613]
[638, 639]
[662, 663]
[669, 670]
[678, 679]
[686, 687]
[705, 706]
[709, 710]
[724, 725]
[740, 741]
[745, 746]
[751, 752]
[760, 761]
[764, 765]
[771, 772]
[774, 776]
[777, 778]
[791, 792]
[796, 798]
[814, 817]
[827, 830]
[845, 849]
[853, 856]
[861, 862]
[865, 86

In [264]:
for elem in kg_sml_dataset:
    for ip, emb in zip(elem["input_ids"], elem["kg_embedding"]):
        print(my_tokenizer.decode(ip), '\t', emb[:5])
    print()

[ C L S ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
[ U N K ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
A l l a h 	 [0.079, 0.1492, -0.125, 0.3824, 0.1091]
( 	 [0.0, 0.0, 0.0, 0.0, 0.0]
/ 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # ˈ 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # æ 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # l 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # ə 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
, 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
ˈ 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # ɑ 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # ː 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # l 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # ə 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
, 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
ə 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # l 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # ˈ 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # l 	 [-0.0475, -0.0114, 0.0836, -0.0178, 0.1227]
# # ɑ 	 [-0.0475, -0.0114, 0.0

, 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
h e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
h a d 	 [0.0, 0.0, 0.0, 0.0, 0.0]
w o r n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a 	 [0.0, 0.0, 0.0, 0.0, 0.0]
r i n g 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h a t 	 [0.0, 0.0, 0.0, 0.0, 0.0]
s a i d 	 [0.0, 0.0, 0.0, 0.0, 0.0]
" 	 [0.0, 0.0, 0.0, 0.0, 0.0]
A l l a h 	 [0.079, 0.1492, -0.125, 0.3824, 0.1091]
i s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
m y 	 [0.0, 0.0, 0.0, 0.0, 0.0]
l o r d 	 [0.0, 0.0, 0.0, 0.0, 0.0]
" 	 [0.0, 0.0, 0.0, 0.0, 0.0]
. 	 [0.0, 0.0, 0.0, 0.0, 0.0]
[ U N K ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
I n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
i n s c r i p t i o n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
o f 	 [0.0, 0.0, 0.0, 0.0, 0.0]
C h r i s t i a n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
m a r t y r 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # i o n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
d a t e d 	 [0.0, 0.0, 0.0, 0.0, 0.0]
b a c k 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t o 	 [0.0, 0.0, 0.0, 0.0, 0.0]
5 1 2 	 [0.0, 0.0, 0.0, 0.0, 0.0]
, 	 [0.0

[ C L S ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
[ U N K ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
T h e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
p o l i t i c s 	 [-0.3241, 0.0879, 0.2027, -0.3456, -0.2076]
o f 	 [-0.3241, 0.0879, 0.2027, -0.3456, -0.2076]
C r o a t i a 	 [-0.3241, 0.0879, 0.2027, -0.3456, -0.2076]
a r e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
d e f i n e d 	 [0.0, 0.0, 0.0, 0.0, 0.0]
b y 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a 	 [0.0, 0.0, 0.0, 0.0, 0.0]
p a r l i a m e n t a r y 	 [-0.5759, 0.2969, -0.0523, -0.4469, -0.4078]
, 	 [0.0, 0.0, 0.0, 0.0, 0.0]
r e p r e s e n t a t i v e 	 [-0.6014, -0.415, -0.0001, -0.0619, -0.7618]
d e m o c r a t i c 	 [-0.6014, -0.415, -0.0001, -0.0619, -0.7618]
r e p u b l i c 	 [-0.5875, 0.1158, -0.2609, 0.1712, -0.1895]
f r a m e w o r k 	 [0.0, 0.0, 0.0, 0.0, 0.0]
, 	 [0.0, 0.0, 0.0, 0.0, 0.0]
w h e r e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
P r i m e 	 [-0.3786, 0.5837, 0.1771, -0.6961, 0.11]
M i n i s t e r 	 [-0.3786, 0.5837, 0.1771, -0.6961, 0.11]
o f 	 [-0.378

# # r s t 	 [-0.305, 0.3632, 0.3082, -0.2726, 0.1272]
# # i 	 [-0.305, 0.3632, 0.3082, -0.2726, 0.1272]
# # č 	 [-0.305, 0.3632, 0.3082, -0.2726, 0.1272]
# # e v i ć 	 [-0.305, 0.3632, 0.3082, -0.2726, 0.1272]
. 	 [0.0, 0.0, 0.0, 0.0, 0.0]
[ U N K ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
G o v e r n m e n t 	 [0.0, 0.0, 0.0, 0.0, 0.0]
m i n i s t e r s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a r e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
f r o m 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
C r o a t i a n 	 [-0.3332, 0.5333, -0.1124, -0.8726, 0.5716]
D e m o c r a t i c 	 [-0.3332, 0.5333, -0.1124, -0.8726, 0.5716]
U n i o n 	 [-0.3332, 0.5333, -0.1124, -0.8726, 0.5716]
( 	 [0.0, 0.0, 0.0, 0.0, 0.0]
H D 	 [-0.3332, 0.5333, -0.1124, -0.8726, 0.5716]
# # Z 	 [-0.3332, 0.5333, -0.1124, -0.8726, 0.5716]
) 	 [0.0, 0.0, 0.0, 0.0, 0.0]
, 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a n d 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h e 	 [-0.3781, 0.191, 0.2441, -0.3133, 0.2165]
B r i d g e 	 [-0.3781, 0.191, 0.2441, -0.3133, 0.2165]
o f 	 [-

# # e a 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # b l e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
. 	 [0.0, 0.0, 0.0, 0.0, 0.0]
" 	 [0.0, 0.0, 0.0, 0.0, 0.0]
[ U N K ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
M o o r e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # y 	 [0.0, 0.0, 0.0, 0.0, 0.0]
w a s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
u n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # c o n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # v i n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # c e d 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a r t i f a c t 	 [0.0, 0.0, 0.0, 0.0, 0.0]
w a s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
i n d e e d 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a 	 [0.0, 0.0, 0.0, 0.0, 0.0]
v a s e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
, 	 [0.0, 0.0, 0.0, 0.0, 0.0]
m e n t i o n i n g 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h a t 	 [0.0, 0.0, 0.0, 0.0, 0.0]
S e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # l i m 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # k h a 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # n o v 	 [0.0, 0.0, 0.0, 0.0, 0.0]
, 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a f t e r 	 [0.0, 0.0, 0.0, 0.0, 0.0]
h i s 	 [0.0, 0.0, 0.0, 0.0, 0

n u c l e a r 	 [0.0, 0.0, 0.0, 0.0, 0.0]
m e m b r a n e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
i s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
c o n t i n u o u s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
w i t h 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
m e m b r a n e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
o f 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
r o u g h 	 [0.581, -0.8189, 0.3056, -0.545, -0.2837]
e n d 	 [0.581, -0.8189, 0.3056, -0.545, -0.2837]
# # o p 	 [0.581, -0.8189, 0.3056, -0.545, -0.2837]
# # l a s 	 [0.581, -0.8189, 0.3056, -0.545, -0.2837]
# # m i c 	 [0.581, -0.8189, 0.3056, -0.545, -0.2837]
r e 	 [0.581, -0.8189, 0.3056, -0.545, -0.2837]
# # t i c 	 [0.581, -0.8189, 0.3056, -0.545, -0.2837]
# # u l u m 	 [0.581, -0.8189, 0.3056, -0.545, -0.2837]
( 	 [0.0, 0.0, 0.0, 0.0, 0.0]
R 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # E R 	 [0.0, 0.0, 0.0, 0.0, 0.0]
) 	 [0.0, 0.0, 0.0, 0.0, 0.0]
, 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a n d 	 [0.0, 0.0, 0.0, 0.0, 0.0]
i s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
s i m i l 

# # i n g 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h e i r 	 [0.0, 0.0, 0.0, 0.0, 0.0]
s t r u c t u r e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
i n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # h i b i t 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t r a n s c r i p t i o n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
o f 	 [0.0, 0.0, 0.0, 0.0, 0.0]
p r o t e i n 	 [0.0, 0.0, 0.0, 0.0, 0.0]
- 	 [0.0, 0.0, 0.0, 0.0, 0.0]
c o d i n g 	 [0.0, 0.0, 0.0, 0.0, 0.0]
g e n e s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
. 	 [0.0, 0.0, 0.0, 0.0, 0.0]
[ U N K ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
L i k e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h e 	 [0.0, 0.0, 0.0, 0.0, 0.0]
c o m p o n e n t s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
o f 	 [0.0, 0.0, 0.0, 0.0, 0.0]
o t h e r 	 [0.0, 0.0, 0.0, 0.0, 0.0]
i n t e r m e d i a t e 	 [0.197, -0.4834, 0.5161, -0.0496, 0.79]
f i 	 [0.197, -0.4834, 0.5161, -0.0496, 0.79]
# # l a m 	 [0.197, -0.4834, 0.5161, -0.0496, 0.79]
# # e n t s 	 [0.197, -0.4834, 0.5161, -0.0496, 0.79]
, 	 [0.0, 0.0, 0.0, 0.0, 0.0]
t h e 	 [0.0, 0.0, 0.0, 0.0, 0

[ C L S ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
[ U N K ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
C 	 [0.1899, 1.1031, 0.6845, -0.6928, -0.1501]
# # a d 	 [0.1899, 1.1031, 0.6845, -0.6928, -0.1501]
# # m i u m 	 [0.1899, 1.1031, 0.6845, -0.6928, -0.1501]
i s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a 	 [0.0, 0.0, 0.0, 0.0, 0.0]
c h e m i c a l 	 [0.1947, 0.5826, 0.5579, -0.6552, -0.403]
e l e m e n t 	 [0.1947, 0.5826, 0.5579, -0.6552, -0.403]
w i t h 	 [0.0, 0.0, 0.0, 0.0, 0.0]
s y m b o l 	 [0.0, 0.0, 0.0, 0.0, 0.0]
C 	 [0.1899, 1.1031, 0.6845, -0.6928, -0.1501]
# # d 	 [0.1899, 1.1031, 0.6845, -0.6928, -0.1501]
a n d 	 [0.0, 0.0, 0.0, 0.0, 0.0]
a t o m i c 	 [0.0551, 0.6858, 0.3534, 0.3378, 0.3772]
n u m b e r 	 [0.0551, 0.6858, 0.3534, 0.3378, 0.3772]
4 8 	 [0.0, 0.0, 0.0, 0.0, 0.0]
. 	 [0.0, 0.0, 0.0, 0.0, 0.0]
[ U N K ] 	 [0.0, 0.0, 0.0, 0.0, 0.0]
T h i s 	 [0.0, 0.0, 0.0, 0.0, 0.0]
s o f t 	 [0.0, 0.0, 0.0, 0.0, 0.0]
, 	 [0.0, 0.0, 0.0, 0.0, 0.0]
b 	 [0.0, 0.0, 0.0, 0.0, 0.0]
# # l u 	 [0.0, 0.0, 0.0, 0.0, 0.0]
#

### Finally!

In [269]:
tokenized_kg_wikitest2_dataset = \
    wikitest2_dataset.map(my_tokenize_function, batched=True)\
                     .map(get_kg_embedding_batched, 
                             batched=True, 
                             batch_size=100, 
                             keep_in_memory=False)

tokenized_kg_wikitest2_dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens', 'kg_embedding'],
        num_rows: 60
    })
})

In [270]:
tokenized_kg_wikitest2_dataset_file = "tokenized_kg_wikitest2_dataset_file"
tokenized_kg_wikitest2_dataset.save_to_disk(tokenized_kg_wikitest2_dataset_file)

In [279]:
# my_tokenizer.all_special_tokens
my_tokenizer("[UNK]") ## 100

{'input_ids': [101, 100, 102], 'attention_mask': [1, 1, 1]}

In [283]:
## what percent of each text is UNK

for row in tokenized_kg_wikitest2_dataset["valid"]["input_ids"]:
    cnt = 0
    for ids in row:
        if ids==100:
            cnt += 1
    print(cnt, '\t', cnt*100/len(row))

53 	 3.5451505016722407
438 	 3.4141398394262996
98 	 3.600293901542983
413 	 3.477894736842105
134 	 3.3242371619945423
56 	 3.9492242595204514
156 	 3.33832655681575
57 	 3.0595813204508855
156 	 3.674046161092793
75 	 3.5919540229885056
73 	 2.7557568893922233
54 	 3.4177215189873418
220 	 3.27917722462364
72 	 3.6345280161534577
153 	 3.8921394047316205
650 	 3.127104782064851
184 	 4.462769827795294
42 	 2.912621359223301
57 	 3.3989266547406083
75 	 3.1446540880503147
93 	 2.9226901319924576
206 	 3.520164046479836
133 	 3.314228756541241
39 	 3.62453531598513
105 	 3.6738978306508048
48 	 4.507042253521127
112 	 3.1390134529147984
80 	 3.6117381489841986
244 	 3.3093720330937204
289 	 3.6494506882182094
197 	 3.0647168637212197
136 	 3.1358081623241874
183 	 2.940231362467866
126 	 3.8508557457212715
241 	 3.54359652992207
57 	 4.136429608127721
176 	 3.5743298131600323
152 	 2.844311377245509
111 	 3.6050665800584607
88 	 3.7130801687763713
238 	 4.490566037735849
56 	 3.172804

In [284]:
tokenized_kg_wikitest2_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens', 'kg_embedding'],
        num_rows: 60
    })
})

In [6]:
tokenized_kg_wikitest2_dataset = load_from_disk("tokenized_kg_wikitest2_dataset_file")

In [7]:
def filter_text_batched(data):
    input_ids_list = data["input_ids"]
    kg_embeds_list = data['kg_embedding']
    
    indices_list = [[i for i,input_id in enumerate(input_ids) if input_id!=100]
                        for input_ids in input_ids_list]
    new_input_ids_list = []
    new_kg_embedding_list = []
        
    for indices, input_ids, kg_embeds in zip(indices_list, input_ids_list, kg_embeds_list):
        
        new_input_ids = [input_ids[ind] for ind in indices]
        new_kg_embeds = [kg_embeds[ind] for ind in indices]
        
        new_input_ids_list.append(new_input_ids)
        new_kg_embedding_list.append(new_kg_embeds)
        
    return {"input_ids": new_input_ids_list, "kg_embedding": new_kg_embedding_list}

filtered_tokenized_kg_wikitest2_dataset = tokenized_kg_wikitest2_dataset.map(filter_text_batched, 
                                             batched=True, 
                                             batch_size=100, 
                                             keep_in_memory=False)
filtered_tokenized_kg_wikitest2_dataset

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens', 'kg_embedding'],
        num_rows: 600
    })
    valid: Dataset({
        features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens', 'kg_embedding'],
        num_rows: 60
    })
    test: Dataset({
        features: ['title', 'tokens', 'annotations', 'input_ids', 'attention_mask', 'word_tokens', 'cummulative_word_tokens', 'kg_embedding'],
        num_rows: 60
    })
})

In [8]:
## what percent of each text is UNK

for row in filtered_tokenized_kg_wikitest2_dataset["valid"]["input_ids"]:
    cnt = 0
    for ids in row:
        if ids==100:
            cnt += 1
    print(cnt, '\t', cnt*100/len(row))

0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0
0 	 0.0


In [10]:
## what percent of each text is UNK

for row in filtered_tokenized_kg_wikitest2_dataset["valid"]["input_ids"]:
    print(len(row))

1442
12391
2624
11462
3897
1362
4517
1806
4090
2013
2576
1526
6489
1909
3778
20136
3939
1400
1620
2310
3089
5646
3880
1037
2753
1017
3456
2135
7129
7630
6231
4201
6041
3146
6560
1321
4748
5192
2968
2282
5062
1709
3836
4611
1429
1407
2258
1897
6434
1422
5908
3059
8184
2928
2023
5716
2245
818
1414
3368


In [11]:
## what percent of each text is UNK

for row in tokenized_kg_wikitest2_dataset["valid"]["input_ids"]:
    cnt = 0
    for ids in row:
        if ids==100:
            cnt += 1
    print(len(row)-cnt)

1442
12391
2624
11462
3897
1362
4517
1806
4090
2013
2576
1526
6489
1909
3778
20136
3939
1400
1620
2310
3089
5646
3880
1037
2753
1017
3456
2135
7129
7630
6231
4201
6041
3146
6560
1321
4748
5192
2968
2282
5062
1709
3836
4611
1429
1407
2258
1897
6434
1422
5908
3059
8184
2928
2023
5716
2245
818
1414
3368


In [9]:
filtered_tokenized_kg_wikitest2_dataset_file = "filtered_tokenized_kg_wikitest2_dataset_file"
filtered_tokenized_kg_wikitest2_dataset.save_to_disk(filtered_tokenized_kg_wikitest2_dataset_file)

In [17]:
chunk_size = 128

def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Loading cached processed dataset at /home/a2khatun/.cache/huggingface/datasets/zhengxuanzenwu___parquet/zhengxuanzenwu--wikitext-2-split-128-f504347a654a9463/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-3fec34e95f2bc86d.arrow


Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 2253
})